In [21]:
import pandas as pd

df = pd.read_csv("공공/훈련데이터셋.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181408 entries, 0 to 181407
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   지역코드           181408 non-null  int64  
 1   최저기온(°C)       181408 non-null  float64
 2   3.0m 지중온도(°C)  181408 non-null  float64
 3   평균 현지기압(hPa)   181408 non-null  float64
 4   가조시간(hr)       181408 non-null  float64
 5   평균 상대습도(%)     181408 non-null  float64
 6   풍정합(100m)      181408 non-null  float64
 7   합계 소형증발량(mm)   181408 non-null  float64
 8   파워             181408 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 12.5 MB


In [22]:
df.head()

,지역코드,최저기온(°C),3.0m 지중온도(°C),평균 현지기압(hPa),가조시간(hr),평균 상대습도(%),풍정합(100m),합계 소형증발량(mm),파워
0,1111010100,-5.2,18.4,1020.9,9.7,38.9,2386.0,1.7,146294.6135
1,1111010100,-7.0,18.4,1018.7,9.7,41.9,2042.0,2.1,175633.8270
2,1111010100,-3.9,18.3,1012.2,9.7,62.8,2104.0,1.9,156084.1910
3,1111010100,0.1,18.2,1015.0,9.7,57.5,1248.0,2.2,177018.8420
4,1111010100,0.9,18.1,1011.7,9.7,60.8,1689.0,1.8,158467.1690


In [23]:

df.describe()

,지역코드,최저기온(°C),3.0m 지중온도(°C),평균 현지기압(hPa),가조시간(hr),평균 상대습도(%),풍정합(100m),합계 소형증발량(mm),파워
count,1.814080e+05,181408.000000,181408.000000,181408.000000,181408.000000,181408.000000,181408.000000,181408.000000,1.814080e+05
mean,1.134053e+09,7.803009,16.931746,1007.101656,11.674789,66.062162,1978.330768,3.480625,1.377028e+05
std,2.008387e+07,11.006211,3.930148,7.849501,1.707213,14.418844,589.293633,2.217958,3.423217e+05
min,1.111010e+09,-17.300000,10.900000,986.100000,9.600000,22.900000,978.000000,0.300000,0.000000e+00
25%,1.114015e+09,-1.500000,13.400000,1001.000000,9.900000,56.100000,1554.000000,1.600000,1.813112e+04
50%,1.129013e+09,8.600000,17.200000,1007.200000,11.600000,66.500000,1871.000000,3.000000,5.471326e+04
75%,1.153010e+09,17.500000,21.000000,1012.700000,13.200000,75.800000,2212.000000,5.000000,1.532241e+05
max,1.174011e+09,27.400000,25.200000,1025.500000,14.800000,99.300000,4990.000000,9.700000,1.269370e+07


In [24]:
target = "파워"  # ← 실제 컬럼명으로 수정
y = df[target]

X = df.drop(columns=[target])
X = X.select_dtypes(include="number")

In [25]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X, y)

r2 = lr.score(X, y)
print("R2:", r2)

R2: 0.051137114510743764


In [28]:
coef_df = pd.DataFrame({
    "feature": X.columns,
    "coef": lr.coef_
}).sort_values("coef", key=abs, ascending=False)

coef_df

,feature,coef
4,가조시간(hr),13400.080591
2,3.0m 지중온도(°C),4713.313107
7,합계 소형증발량(mm),-3413.180847
1,최저기온(°C),-1625.913906
3,평균 현지기압(hPa),-466.820809
5,평균 상대습도(%),-132.839370
6,풍정합(100m),1.163350
0,지역코드,0.003698


In [29]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

vif_df = pd.DataFrame()
vif_df["feature"] = X.columns
vif_df["VIF"] = [
    variance_inflation_factor(X.values, i)
    for i in range(X.shape[1])
]

vif_df.sort_values("VIF", ascending=False)

,feature,VIF
0,지역코드,3162.704064
1,최저기온(°C),9.732265
4,가조시간(hr),6.813584
7,합계 소형증발량(mm),4.188546
5,평균 상대습도(%),3.140494
2,3.0m 지중온도(°C),2.559657
6,풍정합(100m),1.066101
3,평균 현지기압(hPa),0.197464


In [30]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1.0)
ridge.fit(X, y)

ridge_coef = pd.DataFrame({
    "feature": X.columns,
    "coef": ridge.coef_
}).sort_values("coef", key=abs, ascending=False)

ridge_coef

,feature,coef
4,가조시간(hr),13399.868783
2,3.0m 지중온도(°C),4713.277427
7,합계 소형증발량(mm),-3413.137698
1,최저기온(°C),-1625.891216
3,평균 현지기압(hPa),-466.828253
5,평균 상대습도(%),-132.838582
6,풍정합(100m),1.163361
0,지역코드,0.003698


In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R2:", r2)

RMSE: 340878.3290645988
R2: 0.04718565753907955
